In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

input_shape = x_train.shape[1:]

In [4]:
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(32, 32, 3)
)

<ipython-input-5-6934b81f313b>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
num_layers = len(base_model.layers)
print(f"The model has {num_layers} layers.")

The model has 154 layers.


In [6]:
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freeze the first 100 layers
    layer.trainable = False

In [7]:
inputs = keras.Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)  # Set training=False to ensure the base model is in inference mode
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x) # Assuming 10 classes for your task
model = keras.Model(inputs, outputs)

In [8]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 48s 17ms/step - accuracy: 0.1523 - loss: 2.5735 - val_accuracy: 0.1832 - val_loss: 2.2855
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.2849 - loss: 2.0393 - val_accuracy: 0.3028 - val_loss: 2.0494
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.3515 - loss: 1.8415 - val_accuracy: 0.4023 - val_loss: 1.7384
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.4026 - loss: 1.6984 - val_accuracy: 0.4469 - val_loss: 1.6053
Epoch 5/10
 658/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.4269 - loss: 1.6132

KeyboardInterrupt: 